# MNIST 手写数字识别

本次测验需要学生基于MindSpore的API来快速实现一个简单的深度学习模型，完成手写数字识别的任务。该任务旨在考核学生对使用MindSpore完成深度学习全流程的掌握

## 学习资料
- 虚假的学习资料：
  - 官网教程：https://www.mindspore.cn/tutorials/zh-CN/r2.0/index.html
  - MidnSpore版《动手学深度学习》：https://openi.pcl.ac.cn/mindspore-courses/d2l-mindspore
- 真正的学习资料：
    - 实验内容：https://www.mindspore.cn/tutorials/zh-CN/r2.0/beginner/introduction.html

## 环境配置

- MindSpore 2.0， 安装教程：https://www.mindspore.cn/install
- download，可使用命令`pip install download`安装

> 如本练习以Notebook运行时，完成安装后需要重启kernel才能执行后续代码。


In [64]:
import mindspore
from mindspore import nn
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset

## 处理数据集

MindSpore提供基于Pipeline的[数据引擎](https://www.mindspore.cn/docs/zh-CN/r2.0/design/data_engine.html)，通过[数据集（Dataset）](https://www.mindspore.cn/tutorials/zh-CN/r2.0/beginner/dataset.html)和[数据变换（Transforms）](https://www.mindspore.cn/tutorials/zh-CN/r2.0/beginner/transforms.html)实现高效的数据预处理。

在本次练习中，我们使用Mnist数据集，自动下载完成后，使用`mindspore.dataset`提供的数据变换进行预处理。

In [65]:
# Download data from open datasets
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:03<00:00, 3.34MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


数据下载完成后，获得数据集对象。

In [66]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

【练习一】dataset预处理需指定在某个数据列进行操作，请通过`get_col_names`打印数据集中包含的数据列名，用于后续的数据预处理。

In [67]:
# Exercise 1: print the column names in the dataset.
# Add your code here.
# 使用get_col_names方法获取数据列名
column_names = train_dataset.get_col_names()
# 打印数据列名
print("Column Names in the Dataset:", column_names)

Column Names in the Dataset: ['image', 'label']


MindSpore的dataset使用数据处理流水线（Data Processing Pipeline），需指定map、batch、shuffle等操作。这里我们使用map对图像数据及标签进行变换处理，然后将处理好的数据集打包为大小为64的batch。

【练习二】按照如下步骤，补完图像数据的预处理步骤。

- 图片数据处理：
    1. 原始图片中，每个像素的灰度值在0-255之间，我们需要通过`Rescale`将数值变为0-1之间；
    2. 按照mean=0.1307，std=0.3081，通过`Normalize`对数据进行归一化；
    3. 图像的shape为[height(H), width(W), channel(C)]，通过`HWC2CHW`将shape变为[channel(C), height(H), width(W)]；
- 标签数据处理：
    1. 将数据类型通过`TypeCast`转换为`mindspore.int32`；
- 按照batch size进行批处理

In [68]:
from mindspore.dataset import vision, transforms

def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]
    label_transform = transforms.TypeCast(mindspore.int32)
    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset


In [69]:
# Map vision transforms and batch dataset
train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(test_dataset, 64)

使用`create_tuple_iterator`或`create_dict_iterator`对数据集进行迭代。

【练习三】打印第一个batch中图片的shape和dtype，以及标签的shape和dtype。如上述操作正确，图片的shape应为[64, 1, 28, 28]， 标签的dtype应为Int32。

In [70]:
for image, label in train_dataset.create_tuple_iterator():
    # 打印图像的shape和dtype
    print("Image shape:", image.shape)
    print("Image dtype:", image.dtype)
    # 打印标签的shape和dtype
    print("Label shape:", label.shape)
    print("Label dtype:", label.dtype)
    break

Image shape: (64, 1, 28, 28)
Image dtype: Float32
Label shape: (64,)
Label dtype: Int32


更多细节详见[数据集 Dataset](https://www.mindspore.cn/tutorials/zh-CN/r2.0/beginner/dataset.html)与[数据变换 Transforms](https://www.mindspore.cn/tutorials/zh-CN/r2.0/beginner/transforms.html)。

## 网络构建

`mindspore.nn`类是构建所有网络的基类，也是网络的基本单元。当用户需要自定义网络时，可以继承`nn.Cell`类，并重写`__init__`方法和`construct`方法。`__init__`包含所有网络层的定义，`construct`中包含数据（[Tensor](https://www.mindspore.cn/tutorials/zh-CN/r2.0/beginner/tensor.html)）的变换过程（即[计算图](https://www.mindspore.cn/tutorials/zh-CN/r2.0/advanced/compute_graph.html)的构造过程）。

【练习四】参考如下步骤，完成深度学习网络的搭建

1. (已完成) flattern层，将二维图像矩阵转换为一维向量
2. 全连接层，输入为28x28图片转换为一维向量后的长度，输出为512
3. 非线性激活函数 ReLU
4. 全连接层，输入和输出维度相同
5. 非线性层ReLU
6. 全连接层，输入为上一全连接层的输出维度，输出为分类数

可以通过`print(network)`验证网络结构是否正确。

In [59]:
# Define model
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        # Exercise 4：complete the network
        # self.dense_relu_sequential = nn.SequentialCell(
        # Add your code here.
        # )
        self.dense_relu_sequential=nn.SequentialCell(
            nn.Dense(28*28,512),
            nn.ReLU(),
            nn.Dense(512,512),
            nn.ReLU(),
            nn.Dense(512,10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
print(model)

Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


更多细节详见[网络构建](https://www.mindspore.cn/tutorials/zh-CN/r2.0/beginner/model.html)。

## 模型训练

In [71]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), 1e-2)

在模型训练中，一个完整的训练过程（step）需要实现以下三步：

1. **正向计算**：模型预测结果（logits），并与正确标签（label）求预测损失（loss）。
2. **反向传播**：利用自动微分机制，自动求模型参数（parameters）对于loss的梯度（gradients）。
3. **参数优化**：将梯度更新到参数上。

MindSpore使用函数式自动微分机制，因此针对上述步骤需要实现：

1. 正向计算函数定义。
2. 通过函数变换获得梯度计算函数。
3. 训练函数定义，执行正向计算、反向传播和参数优化。

【练习五】参考如上步骤，完成训练一个step的代码。

In [72]:
# Define forward function
def forward_fn(data, label):
    # Exercise 5.1: implement the forward function and ensure that it returns both the loss and logits.
    # Add your code here.
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss, logits


# Get gradient function
grad_fn = mindspore.value_and_grad(forward_fn,
                                   None,
                                   optimizer.parameters,
                                   has_aux=True)


# Define function of one-step training
def train_step(data, label):
    # Exercise 5.2: complete train step function to calculate gradients and optimize the model.
    # The function should return the loss for current step.
    # Add your code here.
    (loss, _), grads = grad_fn(data, label)
    optimizer(grads)
    return loss


def train(model, dataset):
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

除训练外，我们定义测试函数，用来评估模型的性能。

【练习六】补全代码，使每个epoch可以打印当前的averge loss及accuracy。

In [73]:
def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        # Exercise 6: complete the test fucntion to calculate average loss and accuracy.
        # You need to define the following two variables.
        # test_loss: loss for current step.
        # correct: number of correct predictions for current step.
        # Add your code here.
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()

    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程需多次迭代数据集，一次完整的迭代称为一轮（epoch）。在每一轮，遍历训练集进行训练，结束后使用测试集进行预测。打印每一轮的loss值和预测准确率（Accuracy），可以看到loss在不断下降，Accuracy在不断提高。

In [74]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset)
    test(model, test_dataset, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302736  [  0/938]
loss: 2.289075  [100/938]
loss: 2.262667  [200/938]
loss: 2.173080  [300/938]
loss: 1.975729  [400/938]
loss: 1.448676  [500/938]
loss: 1.111629  [600/938]
loss: 0.871680  [700/938]
loss: 0.594046  [800/938]
loss: 0.754966  [900/938]
Test: 
 Accuracy: 85.0%, Avg loss: 0.521185 

Epoch 2
-------------------------------
loss: 0.411710  [  0/938]
loss: 0.411469  [100/938]
loss: 0.549565  [200/938]
loss: 0.351776  [300/938]
loss: 0.569826  [400/938]
loss: 0.792263  [500/938]
loss: 0.303696  [600/938]
loss: 0.189354  [700/938]
loss: 0.498547  [800/938]
loss: 0.318609  [900/938]
Test: 
 Accuracy: 90.2%, Avg loss: 0.339866 

Epoch 3
-------------------------------
loss: 0.325278  [  0/938]
loss: 0.261219  [100/938]
loss: 0.298562  [200/938]
loss: 0.331630  [300/938]
loss: 0.279014  [400/938]
loss: 0.235020  [500/938]
loss: 0.363579  [600/938]
loss: 0.445685  [700/938]
loss: 0.172381  [800/938]
loss: 0.276153  [900/938]
Test: 
 

更多细节详见[模型训练](https://www.mindspore.cn/tutorials/zh-CN/r2.0/beginner/train.html)。

## 保存模型

模型训练完成后，需要将其参数进行保存。

In [75]:
# Save checkpoint
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")

Saved Model to model.ckpt


## 加载模型

加载保存的权重分为两步：

1. 重新实例化模型对象，构造模型。
2. 加载模型参数，并将其加载至模型上。

【练习七】
1. 通过调用`load_checkpoint`和`load_param_into_net`接口，完成模型权重加载。
2. `load_param_into_net`输出未加载权重的参数列表，为空时代表所有参数均加载成功。通过打印`load_param_into_net`结果，验证是否成功加载权重。

In [76]:
# Instantiate a random initialized model
model = Network()
# Load checkpoint and load parameter to model
# Exercise 7.1: complete the process of loading checkpoint.
param_dict = mindspore.load_checkpoint("model.ckpt")

# Exercise 7.2: check if the checkpoint has been successfully loaded by printing the output of load_param_into_net.
# Add your code here.
param_not_load, _ = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)


[]


加载后的模型可以直接用于预测推理。

【练习八】补全代码，完成模型推理。

【测验结果提交】打印推理预测结果，将结果截图提交。

In [79]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    # Exercise 8: complete the model inference process.
    # Add your code here.
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break

Predicted: "[3 8 4 4 4 1 7 2 1 3]", Actual: "[3 8 4 4 4 1 7 2 1 3]"


更多细节详见[保存与加载](https://www.mindspore.cn/tutorials/zh-CN/r2.0/beginner/save_load.html)。